## Start Capacity

Fabric Capacity is an Azure Resource.  In order to be able to manipulate the capacity we need to leverage the azure management APIs.  If your capacity is suspended and needs to be restored, you acheive this with the resume method of the Azure Resource.

### Notes:
1. Constants this notebook depends on are set globally in the [/services/env.py](/services/env.py) class file.
1. I was unable to find API documentation for this API.  I inferred the capabilities existed by referencing the Power Shell libraries for Power BI Embedded capacities:  https://learn.microsoft.com/en-us/powershell/module/az.powerbiembedded/resume-azpowerbiembeddedcapacity?view=azps-11.5.0

In [ ]:
pip install requests msal

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://management.azure.com/.default'
aadToken = AadService.get_access_token(scope)

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
import requests
import json
from services.env import const

apiUrl = f'https://management.azure.com/subscriptions/{const.subscriptionId}/resourceGroups/{const.resourceGroup}/providers/Microsoft.Fabric/capacities/{const.capacityName}/resume?api-version=2022-07-01-preview'       

apiResponse = requests.post(apiUrl, headers=headers)

#error handling for create capacity
if apiResponse.status_code != 202:
    description = f'Error restarting capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print("Capacity restarted")